In [ ]:
!pip install openai --quiet
!pip install python-telegram-bot==13.7 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.1/490.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import time
import json
import openai
import string
import telegram
import pandas as pd

from glob import glob
from pprint import pprint
from bs4 import BeautifulSoup
from difflib import SequenceMatcher

openai.api_key = ''

def similar(a, b):return SequenceMatcher(None, a, b).ratio()

In [ ]:
bot = ''
bbot = telegram.Bot(token=bot)
bbot

In [ ]:
# title_messages = [{"role": "user", "content": prompt_title}]
# title_response = openai.ChatCompletion.create(
#     model='gpt-4',
#     messages=title_messages
# )
# captivating_title = title_response['choices'][0]['message']['content'].strip()
# print(captivating_title)

In [ ]:
def replace_mayusculas_noruego(texto):
    alfabeto_noruego = "ABCDEFGHIJKLMNOPQRSTUVWXYZÆØÅ"
    alfabeto_noruego_minusculas = "abcdefghijklmnopqrstuvwxyzæøå"
    tabla = str.maketrans(alfabeto_noruego, alfabeto_noruego_minusculas)
    texto_en_minusculas = texto.translate(tabla)
    return texto_en_minusculas

def generate_product_info(titles, show=False, number=None):

    MAX_RETRIES = 5
    results, errors = [], []

    for title in titles:
        prompt = f'Given the Norwegian product title "{title}",' + ''' from an eCommerce store offering a range of products from clothing to accessories, \
first generate a captivating 100-word product description optimized for SEO. Emphasize the unique selling points and benefits of the product. Avoid sentences over 20 words and passive voice usage. \
Based on the original title and the new description, craft a new, appealing Norwegian short title that's not just a literal translation, but SEO-optimized, around 1-5 words and simple. \
The title should resonate with the essence of the product and align with the new description. \
The output format should be in JSON format: {title:<original title>, new title:<new title>, description:<new description>}'''

        for attempt in range(MAX_RETRIES):
            try:
                title_messages = [{"role": "user", "content": prompt}]
                title_response = openai.ChatCompletion.create(
                    # model='gpt-3.5-turbo',
                    # model='gpt-4',
                    model='gpt-4-1106-preview',
                    messages=title_messages
                )
                json_output = title_response['choices'][0]['message']['content'].strip()

                # Verify if it's a valid JSON
                try:
                    decoded = json.loads(json_output)
                    if show:print(decoded)
                    results.append(decoded)
                except json.JSONDecodeError:
                    errors.append(title)
                    # Save errors to a JSON file
                    with open(f'errores_{number}.json', 'w') as f:json.dump(errors, f)

                df = pd.DataFrame(results)
                df.to_csv(f'result_{number}_a.csv', encoding='utf8', sep='\t', index=False)
                break

            except openai.error.OpenAIError as e:
                if "Rate limit" in str(e):
                    if attempt < MAX_RETRIES - 1:time.sleep(21)
                    else:raise
                else:raise

    # Return both results and errors
    return results, errors

In [ ]:
csvs = glob("clot*/*.csv")
df = pd.concat([pd.read_csv(i, low_memory=False).drop(columns=['Image Src', "Variant Image", "Image Position"]) for i in csvs], axis=0)
products_df = df[~df['Title'].isna()][2288:2288+1144]
products_df.head(3)

,Handle,Title,Body (HTML),Vendor,Product Category,Type,Tags,Published,Option1 Name,Option1 Value,...,Variant Weight Unit,Variant Tax Code,Cost per item,Included / Norway,Price / Norway,Compare At Price / Norway,Included / International,Price / International,Compare At Price / International,Status
15330,flying-woven-cosy-walking-shoes,Vevde koselige tursko,"<p><span style=""color: #ff9900;""><strong>👟Funk...",Mooz.no,NaN,Vanlige sko-Kvinner,"Damesko, Sandaler og tøfler-Kvinner, Sko, Vanl...",True,Farge,Svart,...,kg,NaN,NaN,True,NaN,NaN,True,NaN,NaN,active
15362,summer-slippers-for-women,Sommertøfler for kvinner,"<h3><span style=""color: #ff8000;""><strong>🌞<sp...",Mooz.no,NaN,Sandaler og tøfler-Kvinner,"Damesko, discount, Sandaler og tøfler-Kvinner,...",True,Farge,grå,...,kg,NaN,NaN,True,NaN,NaN,True,NaN,NaN,active
15389,womens-soft-leather-cyrstal-womens-zip-shoes,Damesko i Mykt Skinn med Krystall og Glidelås,"<h3 data-mce-fragment=""1""><span style=""color: ...",Mooz.no,NaN,Sandaler og tøfler-Kvinner,"Damesko, discount, Sandaler og tøfler-Kvinner,...",True,Farge,Svart,...,kg,NaN,NaN,True,NaN,NaN,True,NaN,NaN,active


In [ ]:
# Usage
l_products_0 = products_df['Title'].tolist()
print(len(l_products_0))

1144


In [ ]:
# l_products_0.index("Hjemsol™ Ikke klebende Grillmatter med Skjæreboks")

In [ ]:
csvs = glob("resul*y.csv")
print(csvs)
n = len(csvs)
print(n)

dz = pd.concat([pd.read_csv(i, encoding='utf8', sep="\t") for i in csvs])
dz.shape

['result_1_y.csv', 'result_0_y.csv', 'result_2_y.csv', 'result_3_y.csv', 'result_4_y.csv', 'result_5_y.csv', 'result_6_y.csv', 'result_7_y.csv', 'result_8_y.csv', 'result_9_y.csv', 'result_10_y.csv', 'result_11_y.csv']
12


(892, 3)

In [ ]:
# aa = 177
aa = dz.shape[0]
bb = aa + 500
print(aa, bb)

l_products = l_products_0[aa:bb]
try:l_s, errores = generate_product_info(l_products, show=True, number=n)
except:que = bbot.send_message(chat_id=, text='revisar y')

892 1392
{'title': '"Forever in my heart" hengende ornament-2D-effekt', 'new title': 'Hjerteornament med evig glede', 'description': 'Få hjertet til å smelte med vårt "Forever in my heart" hengende ornament! Designet med en unik 2D-effekt, vil dette ornamentet være en vakker tilføyelse til ethvert hjem. Utført med ekspertise og kjærlighet, gir dette ornamentet et personlig preg, og minner deg om de spesielle øyeblikkene som alltid vil være i ditt hjerte. Heng det på treet, i vinduet eller som dekor på en gave, dette ornamentet sprer varme og glede. Skap varige minner og behold evig glede med vårt hjerteornament.'}
{'title': 'Glitter Diamond iPhone-deksel', 'new title': 'Diamant iPhone-deksel', 'description': 'Elsker du glitrende eleganse? Skjem bort din iPhone med vårt Diamant iPhone-deksel. Med sin blendende design, er dette dekselet perfekt for å gi din telefon et sofistikert og luksuriøst utseende. Det slitesterke materialet beskytter mot daglig slitasje og holder telefonen trygg he

In [ ]:
# len(errores)
n

12

In [ ]:
# rr = []
# for i in l_s:
#     llaves = list(i.keys())
#     for llave in llaves:
#         if llave not in rr:rr.append(llave)
# rr

In [ ]:
print(glob("*a.csv"))

['result_1_a.csv', 'result_0_a.csv', 'result_2_a.csv', 'result_3_a.csv', 'result_4_a.csv', 'result_5_a.csv', 'result_6_a.csv', 'result_7_a.csv', 'result_8_a.csv', 'result_9_a.csv', 'result_10_a.csv', 'result_11_a.csv', 'result_12_a.csv']


In [ ]:
# df = pd.DataFrame(l_s)
# n = 3
df = pd.read_csv(f"result_{n}_a.csv", encoding='utf8', sep="\t")
# df = pd.DataFrame(W)
# df.columns = ['Title (Original)', "SEO Optimized Title", "SEO Optimized Content"]
df

,title,new title,description
0,"""Forever in my heart"" hengende ornament-2D-effekt",Hjerteornament med evig glede,"Få hjertet til å smelte med vårt ""Forever in m..."
1,Glitter Diamond iPhone-deksel,Diamant iPhone-deksel,Elsker du glitrende eleganse? Skjem bort din i...
2,Universal Tverrsnor i Nylon,Allsidig Nylon Tverrsnor,Oppdag den allsidige Universal Tverrsnoren i N...
3,Ensfarget bluse i bomull og lin,Delikat bomull-linskjorte,Få et delikat og sofistikert look med vår ensf...
4,USB-opptak mikrofon,Profesjonell USB-opptaksmikrofon,Optimaliser opptakene dine med vår profesjonel...
...,...,...,...
159,Universal 360 ° rattforsterkerknott,360 ° rattforsterkerknott,Vil du oppleve total kontroll over rattet? Da ...
160,Jul Bil Torkare Klistermärken,Julbil Dekaler,Gjør bilen din klar for julen med disse fantas...
161,Sklisikker 4-i-1 telefonmatte for bil,Biltelefonmatte - Sklisikker 4-i-1,Oppgrader kjøreopplevelsen din med vår sklisik...
162,Trådløs Bluetooth-selfiestang,Smart Selfiestang,Fang øyeblikkene dine på en helt ny måte med v...


In [ ]:
# df.columns

In [ ]:
dn1 = df[['title', 'new title', 'description']]
dn1

,title,new title,description
0,"""Forever in my heart"" hengende ornament-2D-effekt",Hjerteornament med evig glede,"Få hjertet til å smelte med vårt ""Forever in m..."
1,Glitter Diamond iPhone-deksel,Diamant iPhone-deksel,Elsker du glitrende eleganse? Skjem bort din i...
2,Universal Tverrsnor i Nylon,Allsidig Nylon Tverrsnor,Oppdag den allsidige Universal Tverrsnoren i N...
3,Ensfarget bluse i bomull og lin,Delikat bomull-linskjorte,Få et delikat og sofistikert look med vår ensf...
4,USB-opptak mikrofon,Profesjonell USB-opptaksmikrofon,Optimaliser opptakene dine med vår profesjonel...
...,...,...,...
159,Universal 360 ° rattforsterkerknott,360 ° rattforsterkerknott,Vil du oppleve total kontroll over rattet? Da ...
160,Jul Bil Torkare Klistermärken,Julbil Dekaler,Gjør bilen din klar for julen med disse fantas...
161,Sklisikker 4-i-1 telefonmatte for bil,Biltelefonmatte - Sklisikker 4-i-1,Oppgrader kjøreopplevelsen din med vår sklisik...
162,Trådløs Bluetooth-selfiestang,Smart Selfiestang,Fang øyeblikkene dine på en helt ny måte med v...


In [ ]:
try:
    dn = df[['title', 'new_title', 'description']]
    dn2 = dn[~dn.new_title.isna()].copy()
    dn2 = dn2.rename(columns={'new_title': 'new title'})
except:dn2 = pd.DataFrame([])
dn2

""


In [ ]:
dn3 = pd.concat([dn1, dn2], axis=0)
dn3 = dn3.sort_index(ascending=True)
dn3['new title'] = dn3['new title'].str.capitalize()
dn3

,title,new title,description
0,"""Forever in my heart"" hengende ornament-2D-effekt",Hjerteornament med evig glede,"Få hjertet til å smelte med vårt ""Forever in m..."
1,Glitter Diamond iPhone-deksel,Diamant iphone-deksel,Elsker du glitrende eleganse? Skjem bort din i...
2,Universal Tverrsnor i Nylon,Allsidig nylon tverrsnor,Oppdag den allsidige Universal Tverrsnoren i N...
3,Ensfarget bluse i bomull og lin,Delikat bomull-linskjorte,Få et delikat og sofistikert look med vår ensf...
4,USB-opptak mikrofon,Profesjonell usb-opptaksmikrofon,Optimaliser opptakene dine med vår profesjonel...
...,...,...,...
159,Universal 360 ° rattforsterkerknott,360 ° rattforsterkerknott,Vil du oppleve total kontroll over rattet? Da ...
160,Jul Bil Torkare Klistermärken,Julbil dekaler,Gjør bilen din klar for julen med disse fantas...
161,Sklisikker 4-i-1 telefonmatte for bil,Biltelefonmatte - sklisikker 4-i-1,Oppgrader kjøreopplevelsen din med vår sklisik...
162,Trådløs Bluetooth-selfiestang,Smart selfiestang,Fang øyeblikkene dine på en helt ny måte med v...


In [ ]:
# n = 1
# df.to_csv(f'result_{n}_x.csv', encoding='utf8', sep='\t', index=False)
dn3.to_csv(f'result_{n}_y.csv', encoding='utf8', sep='\t', index=False)

In [ ]:
# df.rename(columns={'new_title': 'new title 2'}, inplace=True)

In [ ]:
csvs = glob("resul*y.csv")
print(csvs)
# n = len(csvs)
print(n)
dz = pd.concat([pd.read_csv(i, encoding='utf8', sep="\t") for i in csvs])
dz.shape

['result_1_y.csv', 'result_0_y.csv', 'result_2_y.csv', 'result_3_y.csv', 'result_4_y.csv', 'result_5_y.csv', 'result_6_y.csv', 'result_7_y.csv', 'result_8_y.csv', 'result_9_y.csv', 'result_10_y.csv', 'result_11_y.csv', 'result_12_y.csv']
12


(1056, 3)

In [ ]:
dz

,title,new title,description
0,Fotballtreningsbelte,Ultimat treningsbelte,Gi fotballtreningen din en boost med det ultim...
1,Ny type sykkel setepute med ventilasjon,Komfortabel sykkel setepute,Opplev topp komfort på dine sykkelturer med de...
2,Nylon bånd hengende spenne for vannflaske,Fleksibel vannflaskeholder,Få mest mulig ut av dine eventyr med vår fleks...
3,Telt naktbeskyter,Vindbeskyttet telt,"Uansett hva været måtte bringe, hold deg komfo..."
4,Sykkelrør eksplosjonsbeskyttet dekkpute,Sikkerhetsdekkpute,Få en trygg og bekymringsfri sykkeltur med vår...
...,...,...,...
159,Universal 360 ° rattforsterkerknott,360 ° rattforsterkerknott,Vil du oppleve total kontroll over rattet? Da ...
160,Jul Bil Torkare Klistermärken,Julbil dekaler,Gjør bilen din klar for julen med disse fantas...
161,Sklisikker 4-i-1 telefonmatte for bil,Biltelefonmatte - sklisikker 4-i-1,Oppgrader kjøreopplevelsen din med vår sklisik...
162,Trådløs Bluetooth-selfiestang,Smart selfiestang,Fang øyeblikkene dine på en helt ny måte med v...


In [ ]:
dm1 = dz[['title', 'new title', 'description']]
dm1

,title,new title,description
0,Fotballtreningsbelte,Ultimat treningsbelte,Gi fotballtreningen din en boost med det ultim...
1,Ny type sykkel setepute med ventilasjon,Komfortabel sykkel setepute,Opplev topp komfort på dine sykkelturer med de...
2,Nylon bånd hengende spenne for vannflaske,Fleksibel vannflaskeholder,Få mest mulig ut av dine eventyr med vår fleks...
3,Telt naktbeskyter,Vindbeskyttet telt,"Uansett hva været måtte bringe, hold deg komfo..."
4,Sykkelrør eksplosjonsbeskyttet dekkpute,Sikkerhetsdekkpute,Få en trygg og bekymringsfri sykkeltur med vår...
...,...,...,...
159,Universal 360 ° rattforsterkerknott,360 ° rattforsterkerknott,Vil du oppleve total kontroll over rattet? Da ...
160,Jul Bil Torkare Klistermärken,Julbil dekaler,Gjør bilen din klar for julen med disse fantas...
161,Sklisikker 4-i-1 telefonmatte for bil,Biltelefonmatte - sklisikker 4-i-1,Oppgrader kjøreopplevelsen din med vår sklisik...
162,Trådløs Bluetooth-selfiestang,Smart selfiestang,Fang øyeblikkene dine på en helt ny måte med v...


In [ ]:
try:
    dm = dz[['title', 'new_title', 'description']]
    dm2 = dm[~dm.new_title.isna()].copy()
    dm2 = dm2.rename(columns={'new_title': 'new title'})
except:
    dm2 = pd.DataFrame([])
dm2

""


In [ ]:
def replace_mayusculas_noruego(texto):
    if not isinstance(texto, str):
        texto = str(texto)  # Convierte el argumento a cadena de texto si no lo es

    alfabeto_noruego = "ABCDEFGHIJKLMNOPQRSTUVWXYZÆØÅ"
    alfabeto_noruego_minusculas = "abcdefghijklmnopqrstuvwxyzæøå"
    tabla = str.maketrans(alfabeto_noruego, alfabeto_noruego_minusculas)
    texto_en_minusculas = texto.translate(tabla)
    return texto_en_minusculas

In [ ]:
dm3 = pd.concat([dm1, dm2], axis=0)
dm3 = dm3.sort_index(ascending=True)
dm3['new title'] = dm3['new title'].str.capitalize()
dm3.loc[:, 'new title'] = [replace_mayusculas_noruego(i).capitalize() for i in dm3['new title']]
dm3

,title,new title,description
0,Fotballtreningsbelte,Ultimat treningsbelte,Gi fotballtreningen din en boost med det ultim...
0,Roterende vindspinner med hjerte-design i rust...,Vindspinner med hjerte-design,Få vindene til å danse med vår roterende vinds...
0,Korogu Familien - Skogsalv-pynt,Sjarmerende skogsalv-pynt,Få et snev av magi og stil med vår Korogu Fami...
0,Speil metall rotator,Luksuriøs metall speil rotator,Oppgrader ditt hjem med vårt luksuriøse Metall...
0,Knagg til Nakkestøtte i Bil,Komfortabel nakkestøtte knagg,Oppgrader kjøreopplevelsen din med vår Knagg t...
...,...,...,...
385,Timeglass til boliginnredning,Stilig tidtaker,Gjør hjemmet ditt enda mer elegant med vårt st...
386,Håndmalte Dyredekorasjoner,Unike dyremotiver,Gå inn i en verden av håndmalte dyredekorasjon...
387,Firefly luminous ornament,Magisk lysende pyntegjenstand,Bring a touch of enchantment to your home with...
388,Funklende Stjerne 300 LED Vindusgardin Streng Lys,Stjernelys vindusgardin,Lys opp hjemmet ditt med vår fantastiske Funkl...


In [ ]:
dm3.isna().sum()

title          0
new title      0
description    0
dtype: int64

In [ ]:
n

12

In [ ]:
dm3.to_csv(f'result_{n}_Y.csv', encoding='utf8', sep='\t', index=False)

In [ ]:
# df['new title 2'].unique()

In [ ]:
df.iloc[0].to_dict()

{'title': '"Forever in my heart" hengende ornament-2D-effekt',
 'new title': 'Hjerteornament med evig glede',
 'description': 'Få hjertet til å smelte med vårt "Forever in my heart" hengende ornament! Designet med en unik 2D-effekt, vil dette ornamentet være en vakker tilføyelse til ethvert hjem. Utført med ekspertise og kjærlighet, gir dette ornamentet et personlig preg, og minner deg om de spesielle øyeblikkene som alltid vil være i ditt hjerte. Heng det på treet, i vinduet eller som dekor på en gave, dette ornamentet sprer varme og glede. Skap varige minner og behold evig glede med vårt hjerteornament.'}

In [ ]:
# len(df.iloc[0]['SEO Optimized Content'].split(" "))

In [ ]:
glob("*Z.csv")

['result_0_Z.csv',
 'result_1_Z.csv',
 'result_2_Z.csv',
 'result_3_Z.csv',
 'result_4_Z.csv',
 'result_5_Z.csv',
 'result_6_Z.csv']

In [ ]:
lista = ['result_19_X.csv', 'result_12_Y.csv', 'result_6_Z.csv']

In [ ]:
dj = pd.concat([pd.read_csv(i, sep='\t', encoding='utf8', low_memory=False, on_bad_lines='skip') for i in lista])
dj

,title,new title,description
0,Kløestillende beroligende Comfrey-salve,Lindrende comfrey-salve for kløe,Opplev en beroligende lindring med vår Kløesti...
1,Usynlig Øyeløft-Klistremerke,Lett og effektivt øyeløft-klistremerke,Opplev det fantastiske resultatet av et usynli...
2,Til min sjelevenn Glitrende runde kjede,Stilfullt glitrende kjede,Finn ditt hjertes kjærlighet med dette til min...
3,Avtakbare tøfler med to formål,Fleksible ottoman tøfler,Forny føttene dine med våre innovative avtakba...
4,Beholder for å bevare friske urter,Frisk urtbeholder,Oppbevar dine urter på en enkel og effektiv må...
...,...,...,...
1139,Uttrekkbar Knapp for Bukser,Fleksibel buksespenne,Opplev bekvemmeligheten og praktiskheten til v...
1140,Skinnjakke for menn,Elegante mote skinnjakke,Opplev det ultimate innen stil og komfort med ...
1141,Mote vanntett par kvarts armbåndsur,Vanntette kvartsur,Hold deg tidløst stilig med våre mote vanntett...
1142,Kjole med klokkeermer,Elegant klokkeerme kjole,Oppdag den ultimate kombinasjonen av stil og k...


In [ ]:
df.to_csv(f'clothing_store_titles.csv', encoding='utf8', sep='\t', index=False)